In [103]:
import (
    "fmt"
    "strconv"
    "strings"
    "os"
    "regexp"
)

func convert(numbers string) (result []int) {
    re := regexp.MustCompile("\\s+")
    for _, e := range re.Split(numbers, -1) {
        if n, err := strconv.Atoi(e); err == nil {
            result = append(result, n)
        } else {
            panic(err)
        }
    }
    return
}

func newArange(r []int) (*arange) {
    return &arange{dst:r[0], src:r[1], spread:r[2]}
}

type arange struct {
    dst int
    src int
    spread int
}

type mapping struct {
    name string
    ranges []*arange
}

func (m *mapping) follow(i int) int {
    for _, r := range m.ranges {
        if r.src <= i && i < (r.src + r.spread) {
            //fmt.Println(m.name, r, i, " -- ", r.src <= i, i < (r.src + r.spread), "as", (r.dst + (i-r.src)))
            return (r.dst + (i-r.src))
        }
    }
    return i
}

func newMapping(lines string) *mapping {
    var result = mapping{ranges: []*arange{}}
    for i, l := range strings.Split(lines, "\n") {
        if(i == 0) {
            result.name = l
        } else {
            result.ranges = append(result.ranges, newArange(convert(l)))
        }
    }
    return &result
}

func convert_seed(seed int, mappings []*mapping) int {
    s := seed
    for _, m := range mappings {
        s = m.follow(s)
    }
    return s
}

func main() {
    
    b, _ := os.ReadFile(".day5-input.txt")
    input := strings.TrimSpace(string(b))
    
    var parts = strings.Split(input, "\n\n");
    var seed_str, map_strs = parts[0], parts[1:]
    seed_str = seed_str[7:]
    
    var seeds = convert(seed_str)
    
    
    var mappings = []*mapping{}
    for _, map_str := range map_strs {
        mappings = append(mappings, newMapping(map_str))
    }
    
    /* part1 commented out
    for _, s := range seeds {
        for _, m := range mappings {
            s = m.follow(s)
        }
        if s < lowest {
            lowest = s
        }
    }
    fmt.Println(lowest)
    */
    
    lowestch := make(chan int)
    
    for i:= 0; i < len(seeds); i += 2 {
        
        go func(seed_start int, seed_end int) {
            fmt.Println("running routine", seed_start, "to", seed_end)
            var lowest = int(^uint(0) >> 1)
            for j := seed_start; j < seed_start+seed_end; j += 1 {
                s := convert_seed(j, mappings)
                if s < lowest {
                    lowest = s
                }
            }
            lowestch <- lowest
        }(seeds[i], seeds[i+1])
    }
    
    var lowest = int(^uint(0) >> 1)
    for i := 0; i < len(seeds)/2; i++ {
        res, _ := <-lowestch
        if res < lowest {
            lowest = res
        }
    }
    close(lowestch)
    
    fmt.Println(lowest)
}

running routine 2102534948 to 5923540
running routine 2249227634 to 74967914
running routine 658370118 to 49359719
running routine 4055197159 to 59237418
running routine 314462259 to 268880047
running routine 3793791524 to 105394212
running routine 1514493331 to 295250933
running routine 2370414906 to 38444198
running routine 3291001718 to 85800943
running routine 828589016 to 654882197
7873084
